In [1]:
!pip install openpyxl
!pip install matplotlib
!pip install seaborn

import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms
import torchvision.models as models
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import data
brain_slices = np.load('/rds/projects/b/bowmanh-msc-project-group-2023to2024/Yifei_/revised_stitched_ws_images_v5.npy',allow_pickle=True)
df = pd.read_excel('/rds/projects/b/bowmanh-msc-project-group-2023to2024/Yifei_/Patients_CathyExport_processed_v5.xlsx', sheet_name='master_run')

In [3]:
patient_data_nn = df[:1072]

In [4]:
scanners_to_exclude = ['SON', 'NCL', 'EXT', 'UCH']
training_list = [1,2,3,4]
test_list = [5]
s_train_val = brain_slices[patient_data_nn.NewGroups.isin(training_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]
s_test = brain_slices[patient_data_nn.NewGroups.isin(test_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]

print("X_train_val:", s_train_val.shape)
print("X_test:", s_test.shape)

X_train_val: (608, 632, 760)
X_test: (150, 632, 760)


In [5]:
labels_train = patient_data_nn[patient_data_nn.NewGroups.isin(training_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]['SpkPicDescTOTTS']
labels_test = patient_data_nn[patient_data_nn.NewGroups.isin(test_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]['SpkPicDescTOTTS']

y_train_val = (labels_train>=60).astype(int)
y_test = (labels_test>=60).astype(int)

print("y_train_val:", len(y_train_val))
print("y_test:", len(y_test))

y_train_val: 608
y_test: 150


In [6]:
# Preprocess the image data
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
#     transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_slices(slices):
    tensors = [preprocess(torch.tensor(slice, dtype=torch.float).unsqueeze(0)) for slice in slices]
    return torch.stack(tensors)

X_train_test = preprocess_slices(s_train_val).to(device)
y_train_test = torch.from_numpy(y_train_val.values).long().to(device)
X_test = preprocess_slices(s_test).to(device)
y_test = torch.from_numpy(y_test.values).long().to(device)

print(f"Number of Training/Validation: {len(y_train_test)}")
print(f"Number of Testing: {len(y_test)}")

# Preprocess the image data (assuming preprocess_slices function is defined)
print(f"Train/Validation Images Shape: {X_train_test.shape}")
print(f"Test Images Shape: {X_test.shape}")

test_data = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

Number of Training/Validation: 608
Number of Testing: 150
Train/Validation Images Shape: torch.Size([608, 3, 256, 307])
Test Images Shape: torch.Size([150, 3, 256, 307])


In [7]:
# Define the model
class_names = ['below','above']

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(256, len(class_names))
)

model = model.to(device)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

/rds/homes/y/yxm326/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/rds/homes/y/yxm326/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Earlystopping strateges:
#### 1. UP for α length
#### 2. Generalization Loss over α
#### 3. improvement of 2
#### 4. cross
#### 5. run for 200 epochs for minimum

In [8]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [9]:
class EarlyStoppingAvgLoss:
    def __init__(self, patience=5, avg_len=10):
        self.patience = patience
        self.avg_len = avg_len
        self.best_avg_loss = float('inf')
        self.losses = []
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        self.losses.append(val_loss)
        if len(self.losses) > self.avg_len:
            self.losses = self.losses[-self.avg_len:]
        avg_loss = sum(self.losses) / len(self.losses)
        if avg_loss < self.best_avg_loss:
            self.best_avg_loss = avg_loss
            self.counter = 0
        else:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")

            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [10]:
class EarlyStoppingWeight:
    def __init__(self, patience=5, alpha=0.5):
        self.patience = patience
        self.alpha = alpha
        self.best_avg_loss = float('inf')
        self.avg_loss = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.avg_loss == 0:
            self.avg_loss = val_loss
        else:
            self.last_loss = self.avg_loss
            self.avg_loss = self.alpha * val_loss + (1 - self.alpha) * self.avg_loss
        if self.avg_loss < self.best_avg_loss:
            self.best_avg_loss = self.avg_loss
        elif val_loss < self.last_loss:
            self.counter = 0
        else:
            self.counter += 1
#             print(f"INFO: Early stopping counter {self.counter} of {self.patience}")

            if self.counter >= self.patience:
#                 print('INFO: Early stopping')
                self.early_stop = True

### GL

In [11]:
class EarlyStoppingGL:
    def __init__(self, alpha=0.5):
        """
        Initialize the early stopping criteria based on Generalization Loss (GL).

        Parameters:
        alpha (float): Threshold for the GL to trigger early stopping.
        """
        self.alpha = alpha
        self.best_loss = float('inf')
        self.early_stop = False
        self.stop_epoch = None
        self.val_losses = []

    def GL(self, val_loss, min_val_loss):
        return 100 * (val_loss / min_val_loss - 1)

    def __call__(self, val_loss):
        self.val_losses.append(val_loss)
        if val_loss < self.best_loss:
            self.best_loss = val_loss

        GL_value = self.GL(val_loss, self.best_loss)
        if GL_value > self.alpha:
            self.early_stop = True
            self.stop_epoch = len(self.val_losses)

In [12]:
class EarlyStoppingGLPQRatio:
    def __init__(self, alpha=0.95, k=5):
        """
        Initialize the early stopping criteria based on GL/PQ ratio.

        Parameters:
        alpha (float): Threshold for the GL/PQ ratio to trigger early stopping.
        k (int): Number of epochs over which to calculate the PQ value.
        """
        self.alpha = alpha
        self.k = k
        self.best_loss = float('inf')
        self.val_losses = []
        self.train_losses = []
        self.early_stop = False
        self.stop_epoch = None

    def GL(self, val_loss, min_val_loss):
        return 1000 * (val_loss / min_val_loss - 1)

    def PQ(self, train_losses):
        min_train_loss = min(train_losses)
        avg_train_loss = sum(train_losses) / len(train_losses)
        return 100 * (avg_train_loss / min_train_loss - 1)

    def __call__(self, val_loss, train_loss):
        self.val_losses.append(val_loss)
        self.train_losses.append(train_loss)

        if val_loss < self.best_loss:
            self.best_loss = val_loss

        else:
            GL_value = self.GL(val_loss, self.best_loss)
            PQ_value = self.PQ(self.train_losses[-self.k:])
            if GL_value / PQ_value > self.alpha:
                self.early_stop = True
                self.stop_epoch = len(self.val_losses)

In [13]:
class EarlyStoppingValLoss:
    def __init__(self, s=20):
        """
        Initialize the early stopping criteria based on consecutive increases in validation loss.

        Parameters:
        s (int): Number of consecutive epochs with increasing validation loss to trigger early stopping.
        """
        self.s = s
        self.best_loss = float('inf')
        self.val_losses = []
        self.early_stop = False
        self.stop_epoch = None

    def __call__(self, val_loss):
        """
        Check if training should be stopped based on consecutive increases in validation loss.

        Parameters:
        val_loss (float): Current validation loss.
        """
        self.val_losses.append(val_loss)
        if val_loss < self.best_loss:
            self.best_loss = val_loss

        if len(self.val_losses) > self.s:
            increasing = all(x < y for x, y in zip(self.val_losses[-self.s:], self.val_losses[-self.s + 1:]))
            if increasing:
                self.early_stop = True
                self.stop_epoch = len(self.val_losses)

In [14]:
best_val_loss = float('inf')   # Initialize the best validation loss to positive infinity
model_path = 'best_model.pth'  # Path to save the best model weights
n_epochs = 50
# early_stopping = EarlyStopping(patience=20, min_delta=0.01)
# early_stopping = EarlyStoppingAvgLoss(patience=20, avg_len=3)
# early_stopping = EarlyStoppingEWMA(patience=10, alpha=0.5)

# early_stopping_gl = EarlyStoppingGL()
# early_stopping_glqr = EarlyStoppingGLPQRatio()
# early_stopping_val_loss = EarlyStoppingValLoss()

In [15]:
import csv
import collections
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.model_selection import KFold

# Assume necessary imports and definitions (EarlyStoppingWeight, device, class_names, etc.)

def create_model():
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, len(class_names))
    )
    return model.to(device)

def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        predicted = (output > 0).float()
        total_train += target.size(0) * target.size(1)
        correct_train += (predicted == target).float().sum()
    
    return train_loss, correct_train, total_train

def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
            
            output = model(data)
            loss = criterion(output, target)
            
            val_loss += loss.item() * data.size(0)
            predicted = (output > 0).float()
            total_val += target.size(0) * target.size(1)
            correct_val += (predicted == target).float().sum()
    
    return val_loss, correct_val, total_val

def train_and_validate(X_train, y_train, X_val, y_val, alpha, patience, n_epochs):
    model = create_model()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)
    early_stopping = EarlyStoppingWeight(patience=patience, alpha=alpha)
    
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_val, y_val), batch_size=32)
    
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_epoch = None
    
    for epoch in range(n_epochs):
        train_loss, correct_train, total_train = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, correct_val, total_val = validate(model, val_loader, criterion)
        
        train_accuracy = 100 * correct_train / total_train
        val_accuracy = 100 * correct_val / total_val
        
        train_losses.append(train_loss / len(train_loader.dataset))
        val_losses.append(val_loss / len(val_loader.dataset))
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        
        early_stopping(val_losses[-1])
        if early_stopping.early_stop:
            print(f"    Early stopping at epoch {epoch + 1}")
            best_epoch = epoch
            break

    if best_epoch == None:
        best_epoch = n_epochs - 1

    print(f'    Val Accuracy: {val_accuracies[best_epoch]:.2f}%')
    
    return model, train_losses[best_epoch], val_losses[best_epoch], train_accuracies[best_epoch], val_accuracies[best_epoch]

def test_model(model, X_test, y_test):
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=32)
    criterion = nn.BCEWithLogitsLoss()
    _, correct_test, total_test = validate(model, test_loader, criterion)
    test_accuracy = 100 * correct_test / total_test
    return test_accuracy

def run_experiment(X_train_test, y_train_test, alpha_values, patience_values, num_runs, outer_k, inner_k, n_epochs):
    all_val_accuracies = [[[] for _ in range(len(alpha_values) * len(patience_values))] for _ in range(num_runs)]

    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs}")
        outer_kfold = KFold(n_splits=outer_k, shuffle=False)
        best_val_loss = float('inf')
        best_combination = None
        best_val_accuracy = None

        for outer_fold, (train_val_idx, test_idx) in enumerate(outer_kfold.split(X_train_test)):
            print(f'Outer Fold {outer_fold + 1}')
            X_train_val, y_train_val = X_train_test[train_val_idx], y_train_test[train_val_idx]
            X_test, y_test = X_train_test[test_idx], y_train_test[test_idx]

            inner_kfold = KFold(n_splits=inner_k, shuffle=False)
            train_idx, val_idx = next(inner_kfold.split(X_train_val))
            X_train, y_train = X_train_val[train_idx], y_train_val[train_idx]
            X_val, y_val = X_train_val[val_idx], y_train_val[val_idx]

            for alpha_idx, alpha in enumerate(alpha_values):
                for patience_idx, patience in enumerate(patience_values):
                    print(f'  Alpha {alpha}, Patience {patience}')
                    model, _, val_loss, _, val_accuracy = train_and_validate(X_train, y_train, X_val, y_val, alpha, patience, n_epochs)
                    all_val_accuracies[run][alpha_idx * len(patience_values) + patience_idx].append(val_accuracy)
                    
                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        best_combination = (alpha, patience)
                        best_val_accuracy = val_accuracy

        print("=============== Info ===================")
        print(f'Best Val Loss: {best_val_loss:.4f}')
        if best_val_accuracy != None and best_combination != None:
            print(f'Val Accuracy in Best Val Loss: {best_val_accuracy:.2f}%')
            print(f'Combination in Best Val Loss: Alpha {best_combination[0]}, Patience {best_combination[1]}')
        print("=============== End ===================")

    return all_val_accuracies

def write_results_to_csv(all_val_accuracies, alpha_values, patience_values, num_runs, outer_k):
    with open('experiment_results.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        # Write header
        header = ['Run', 'Outer Fold', 'Alpha', 'Patience', 'Test Accuracy']
        csvwriter.writerow(header)
        
        # Write data
        for run in range(num_runs):
            for outer_fold in range(outer_k):
                for alpha_idx, alpha in enumerate(alpha_values):
                    for patience_idx, patience in enumerate(patience_values):
                        combo_idx = alpha_idx * len(patience_values) + patience_idx
                        accuracy = all_val_accuracies[run][combo_idx][outer_fold]
                        row = [run + 1, outer_fold + 1, alpha, patience, f"{accuracy:.2f}"]
                        csvwriter.writerow(row)
        
        print("Results have been written to experiment_results.csv")
        
# Main execution
if __name__ == "__main__":
    # Define hyperparameters
    num_runs = 20
    alpha_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    patience_values = [5, 6, 7, 8, 9, 10]
    outer_k = 4
    inner_k = 3
    n_epochs = 50

    all_outer_accuracies = run_experiment(
        X_train_test, y_train_test, alpha_values, patience_values, num_runs, outer_k, inner_k, n_epochs
    )

    # Write results to CSV
    write_results_to_csv(all_outer_accuracies, alpha_values, patience_values, num_runs, outer_k)

Run 1/20
Outer Fold 1
  Alpha 0.1, Patience 5
    Val Accuracy: 69.74%
  Alpha 0.1, Patience 6
    Val Accuracy: 67.76%
  Alpha 0.1, Patience 7
    Early stopping at epoch 50
    Val Accuracy: 67.43%
  Alpha 0.1, Patience 8
    Early stopping at epoch 49
    Val Accuracy: 65.79%
  Alpha 0.1, Patience 9
    Val Accuracy: 68.42%
  Alpha 0.1, Patience 10
    Val Accuracy: 68.42%
  Alpha 0.2, Patience 5
    Val Accuracy: 70.39%
  Alpha 0.2, Patience 6
    Early stopping at epoch 49
    Val Accuracy: 68.09%
  Alpha 0.2, Patience 7
    Early stopping at epoch 47
    Val Accuracy: 71.38%
  Alpha 0.2, Patience 8
    Early stopping at epoch 48
    Val Accuracy: 67.76%
  Alpha 0.2, Patience 9
    Val Accuracy: 67.43%
  Alpha 0.2, Patience 10
    Early stopping at epoch 50
    Val Accuracy: 64.47%
  Alpha 0.3, Patience 5
    Early stopping at epoch 41
    Val Accuracy: 66.12%
  Alpha 0.3, Patience 6
    Early stopping at epoch 46
    Val Accuracy: 69.74%
  Alpha 0.3, Patience 7
    Val Accuracy: 

# End